**<font size="15" color="gray">Prosetta </font>**  <font size="7" color="magenta"> *Plate Viewer*</font> 

# Code

<font size=4, color="red">**Warning : **</font>
The stuff in the following sections is the code that creates the graph. Don't change any of this unless you really know what you are doing.

If you *MUST* change the code, click the eyeball icon to view the source code.

## Import Packages and Data

In [1]:
%pylab inline
#%matplotlib notebook

Populating the interactive namespace from numpy and matplotlib


In [2]:
import openpyxl
import sys
import os
from brewer2mpl import qualitative
#from matplotlib import cm  #<-- this package blows, so i used brewer
#from matplotlib.patches import Rectangle #<-- failed to get this to work

In [3]:
# this cell imports packages for ipywidgets
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from IPython.display import display

In [4]:
# this function gets all the filenames
def getFiles():
    data_dir = "./data"
    files = os.listdir(data_dir)
    return [v for i,v in enumerate(files) if ".xlsx" in v]

## Helper Functions and Classes

### Helper Functions

In [5]:
def getBox(min_row, max_row, min_col, max_col): #<-- refer to (row,col) of "name" cell
    box = []
    for row in ws.iter_rows( min_row=min_row, max_row=max_row, min_col=min_col, max_col=max_col ):
        a_row = [] 
        for cell in row:
            a_row.append(cell.value)
        box.append( a_row )
    return np.array(box)

In [6]:
def getDatapoints( name_pos, ): #<-- refer to (row,col) of "name" cell
    row,col = name_pos[0], name_pos[1]
    return getBox(min_row=row-9, max_row=row-6, min_col=col-1, max_col=col+1)

def getRatios( name_pos ): #<-- refer to (row,col) of "name" cell
    row,col = name_pos[0], name_pos[1]
    ratios_raw =  getBox(min_row=row, max_row=row+3, min_col=col-1, max_col=col-1).flatten()
    
    # If any of the raw ratios are empty we set it's value to the corresponding global ratio
    ratios_final = []
    for i in range(len(ratios_raw)):
        if ratios_raw[i] == None:
            ratios_final.append( ratios[i] ) #<-- ratios is a global variable
        else: 
            ratios_final.append( ratios_raw[i] )
    return np.array( ratios_final )



def getControls(row=6,col=11):   #<-- refer row,cell of upper left corner of control box
    c1 = getBox(min_row=row, max_row=row+1, min_col=col, max_col=col+2).flatten()
    c2 = getBox(min_row=row+2, max_row=row+2, min_col=col, max_col=col+2).flatten()
    c3 = getBox(min_row=row+3, max_row=row+3, min_col=col, max_col=col+2).flatten()
    return [c1,c2,c3]

### Helper Classes

Equation used to modify datapoints in data fixed:
$$\frac{\text{values}-\text{mean}(c_{3})}{\text{fixed_mean}(c_{1})}\times100$$

In [7]:
class Compound:
    def __init__(self, name_pos, controls ):
        self.name_pos = name_pos
        
        self.name   = str( ws.cell(row=name_pos[0], column=name_pos[1]).value)
        self.data   = getDatapoints(name_pos)
        self.ratios = getRatios( name_pos)
        
        # we now order data according to concentrations 
        idx = np.argsort( self.ratios )
        self.data   = self.data[  idx]
        self.ratios = self.ratios[idx]
        
        self.data_fixed = (self.data - controls.means[2]) / controls.fixed_means[0] * 100
        self.means = self.data_fixed.mean( axis=1 )
        self.sds = self.data_fixed.std( axis=1,ddof=1 )

In [8]:
class Controls:
    def __init__(self, row=6,col=11):
        c1 = getBox(min_row=row, max_row=row+1, min_col=col, max_col=col+2).flatten()
        c2 = getBox(min_row=row+2, max_row=row+2, min_col=col, max_col=col+2).flatten()
        c3 = getBox(min_row=row+3, max_row=row+3, min_col=col, max_col=col+2).flatten()

        self.data  = [c1,c2,c3]
        self.means = np.array([ x.mean() for x in self.data ]) 
        self.sds   = np.array([ x.std(ddof=1) for x in self.data ]) 
        
        self.fixed_means = self.means - self.means[2]

### The Mother Class

In [9]:
class Plate:
        
    def __init__(self, filename):
        # we make hte worksheet and workbook global so that the Complound and Controls classes can access them
        global ws
        global wb
        self.filename = filename
        wb = openpyxl.load_workbook("data/" + filename )   #<-- open excel workbook
        ws = wb.get_sheet_by_name("Sheet1")                #<-- focus in on first sheet
        
        # This allows the Compound class to grab default ratios non are provided
        global ratios
        ratios= getRatios( (11,3) )
        
        # Make controls object
        self.temp = int( ws["A2"].value)  
        self.controls = Controls()
        
        # Make compounds objects
        name_spots = [(11,3),(15,3),(11,6),(15,6),(11,9),(15,9),(11,12)]  #<- hard code where to look for names
        self.compounds =[]
        for name_pos in name_spots:
            self.compounds.append( Compound(name_pos, self.controls) )

            
            
    # this code does the plotting
    def makePlot(self):
        plt.rcParams["figure.figsize"]=[14,7]

        n_groups = len(self.compounds)
        index = np.arange(0,n_groups)
        bar_width = 0.22

        opacity, error_config = 0.7, {'ecolor': '0.2'}
        colors = qualitative.Dark2[4].mpl_colors #colors  = ["g","m","y","b"]
        
        for i in range(4):
            plt.bar(index+bar_width*i,    #<-- this creates all the bars of one color
                    [x.means[i] for x in self.compounds], #<-- the height of the bar, 
                    bar_width,
                    alpha=opacity,
                    color=colors[i],
                    yerr=[x.sds[i] for x in self.compounds],  #<-- the SD of the bar
                    error_kw=error_config)

            for j in index:   #<-- this creates the bars sub labels
                plt.text( j+bar_width*i-.07, -3.5, self.compounds[j].ratios[i] )



        plt.title( self.filename[:-5], fontsize=24)
        plt.xticks(index + 1.5*bar_width/2+.2 , ["\n"+x.name for x in self.compounds] ,  fontsize=12 )

        # add final bar indicating control
        plt.bar([-.3], 100, width=.2) 
        #plt.Rectangle([0,0],100,.2, facecolor="b")
        plt.text( -.35, -3.5, "D" )

        plt.grid(True, axis="y", alpha=.3)
        plt.tight_layout()
        plt.legend()   #<-- this is not for legen, it just removes some awkward printout before graph 
        plt.show()

# View Plates

Use the tools below to look at the plot for any of the plates whose excel file has been uploaded to the [data file](http://70.95.166.175:8888/tree/Yemi/data) . Click the icon at the top of the screen that looks like a bar chart to enter <font color="blue">fullscreen mode</font>. <font color="gray">(the button is right underneath "help")</font>

Inside the [data file](http://70.95.166.175:8888/tree/Yemi/data), <font color="green">Template.xlsx</font> is an example you can download to see how you should format your data. To view new files added to the [data file](http://70.95.166.175:8888/tree/Yemi/data), **update the program** by clicking on the cell below and then pressing **`shift+enter`**.

In [10]:
@interact(filename= getFiles() )
def getPlot( filename ):
    plate = Plate(filename)
    plate.makePlot()

A Jupyter Widget